In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [28]:
# Загружаем преобработанные файлы для train

df_orders_train = pd.read_parquet('data\\train\\df_orders_flat.parquet')
df_visits_train = pd.read_parquet('data\\train\\df_visits_flat.parquet')
df_last_visits_cat_train = pd.read_parquet('data\\train\\df_last_visits_cat_flat.parquet')

In [29]:
# Загружаем преобработанные файлы для val

df_orders_val = pd.read_parquet('data\\val\\df_orders_flat.parquet')
df_visits_val = pd.read_parquet('data\\val\\df_visits_flat.parquet')
df_last_visits_cat_val = pd.read_parquet('data\\val\\df_last_visits_cat_flat.parquet')

In [30]:
df_visits_train

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_1,female,3,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,0.0,1.0,None,None,None
2,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,112.0,2.0,None,None,None
3,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None
4,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.694267e+09,0.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2111233,user_127750,male,11,578.0,1.692981e+09,1.692981e+09,1.692981e+09,0.0,1.0,None,None,None
2111234,user_127750,male,11,371.0,1.691270e+09,1.691270e+09,1.691270e+09,32.0,4.0,[item_5409132],None,None
2111235,user_127751,male,11,3.0,1.697627e+09,1.697627e+09,1.697627e+09,0.0,1.0,None,None,None
2111236,user_127751,male,11,223.0,1.697627e+09,1.697627e+09,1.697627e+09,5.0,2.0,None,None,None


In [31]:
df_visits_val

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,None,None,None
1,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697133e+09,0.0,1.0,None,None,None
2,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698855e+09,626.0,3.0,None,None,None
3,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698931e+09,0.0,1.0,None,None,None
4,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698947e+09,0.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,None,None,None
561074,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,None,None,None
561075,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,None,None,None
561076,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,None,None,None


In [32]:
df_visits_common = pd.concat([df_visits_train, df_visits_val])
df_visits_common

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_1,female,3,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,0.0,1.0,None,None,None
2,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,112.0,2.0,None,None,None
3,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None
4,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.694267e+09,0.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,None,None,None
561074,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,None,None,None
561075,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,None,None,None
561076,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,None,None,None


In [33]:
df_visits_common = df_visits_common[df_visits_common['site-id'] != 3]
df_visits_common

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_1,female,3,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
6,user_5,female,35,23.0,1.690603e+09,1.690603e+09,1.690603e+09,115.0,2.0,"[item_18, item_19]",None,None
7,user_5,female,35,13.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None
8,user_5,female,35,13.0,1.693797e+09,1.698664e+09,1.693798e+09,582.0,6.0,None,None,None
9,user_5,female,35,13.0,1.693797e+09,1.698664e+09,1.693811e+09,0.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
561070,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.691665e+09,211.0,3.0,"[item_4410808, item_4410807, item_4410806]","[91203, 15625443, 7811879, 91197, 7877999, 156...","[3432476, 3430741, 3436059]"
561071,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.691736e+09,0.0,1.0,[item_172267],"[91197, 91210, 90801]",[3429210]
561072,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.691751e+09,0.0,1.0,[item_751541],"[7811879, 7877999, 15625452, 15625409]",[3439589]
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,None,None,None


In [34]:
male_site_id_stat = df_visits_common[df_visits_common['target'] == 'male']['site-id'].value_counts(normalize=True) * 100
male_site_id_stat = dict(male_site_id_stat)
male_site_id_stat[3.0] = 0
male_site_id_stat[0.0] = 0
male_site_id_stat

{20.0: 10.50393263973447,
 16.0: 6.195376738894347,
 41.0: 4.191023332565454,
 29.0: 4.136582246940628,
 23.0: 3.8003450756204544,
 2.0: 3.442858015956222,
 13.0: 3.292980484892364,
 7.0: 2.190995124749068,
 153.0: 1.9162509931502019,
 206.0: 1.8678275750210385,
 33.0: 1.7708867127080914,
 37.0: 1.768536061342598,
 21.0: 1.4734823019458694,
 371.0: 1.472542041399672,
 578.0: 1.358864541364412,
 19.0: 1.3540692125788056,
 30.0: 1.2198000065818237,
 62.0: 1.1505968303816987,
 34.0: 1.1341422708232451,
 49.0: 1.106498610765043,
 307.0: 1.0825219668370105,
 98.0: 1.0569468799804427,
 147.0: 1.0398341380396507,
 39.0: 0.8883581640472575,
 65.0: 0.883562835261651,
 8.0: 0.7230603600257631,
 10.0: 0.6941003352028847,
 42.0: 0.6935361788751663,
 46.0: 0.6779278538082902,
 208.0: 0.6754831763881771,
 348.0: 0.6715340820941482,
 216.0: 0.6380608066495226,
 40.0: 0.6352400250109305,
 100.0: 0.5994160982008114,
 31.0: 0.573652959235004,
 26.0: 0.5255116192696996,
 22.0: 0.5143225187699512,
 234.0:

In [35]:
female_site_id_stat = df_visits_common[df_visits_common['target'] == 'female']['site-id'].value_counts(normalize=True) * 100
female_site_id_stat = dict(female_site_id_stat)
female_site_id_stat[3.0] = 0
female_site_id_stat[0.0] = 0
female_site_id_stat

{20.0: 7.216578558542721,
 16.0: 4.225004290300572,
 13.0: 3.6389283038453653,
 29.0: 2.8663393481254524,
 7.0: 2.7612793048457376,
 34.0: 2.7082051475235547,
 41.0: 2.6333655629083634,
 8.0: 2.1685899770207318,
 62.0: 1.9701060227448715,
 371.0: 1.7418201757139689,
 65.0: 1.7248263997890427,
 98.0: 1.703898104314996,
 46.0: 1.6702454051927287,
 2.0: 1.5857788046594756,
 208.0: 1.3795095244672702,
 23.0: 1.333885840333848,
 49.0: 1.333048708514886,
 42.0: 1.318566328046846,
 206.0: 1.2116645947654148,
 199.0: 1.1991076174809867,
 37.0: 1.192159423383603,
 10.0: 1.1323044983278292,
 21.0: 1.121840350590806,
 33.0: 1.110790210580509,
 209.0: 1.080402325552193,
 31.0: 1.062069138716928,
 307.0: 1.0007073763870227,
 175.0: 0.9625341654423614,
 12.0: 0.9389270481476366,
 578.0: 0.8557998585247226,
 407.0: 0.8287605007722542,
 100.0: 0.7966983521060144,
 97.0: 0.7867364834603682,
 118.0: 0.7675661648061413,
 93.0: 0.7061206892943398,
 149.0: 0.6855272465478777,
 58.0: 0.6841041224556424,
 51

In [36]:
df_visits_train['site-id'] = df_visits_train['site-id'].fillna(0)
df_visits_train

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_1,female,3,0.0,NaN,NaN,NaN,NaN,NaN,None,None,None
1,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,0.0,1.0,None,None,None
2,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,112.0,2.0,None,None,None
3,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None
4,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.694267e+09,0.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2111233,user_127750,male,11,578.0,1.692981e+09,1.692981e+09,1.692981e+09,0.0,1.0,None,None,None
2111234,user_127750,male,11,371.0,1.691270e+09,1.691270e+09,1.691270e+09,32.0,4.0,[item_5409132],None,None
2111235,user_127751,male,11,3.0,1.697627e+09,1.697627e+09,1.697627e+09,0.0,1.0,None,None,None
2111236,user_127751,male,11,223.0,1.697627e+09,1.697627e+09,1.697627e+09,5.0,2.0,None,None,None


In [38]:
df_visits_train['male_site_id'] = df_visits_train['site-id'].apply(lambda x: male_site_id_stat[x] if x in male_site_id_stat else 0.0)
df_visits_train

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands,male_site_id
0,user_1,female,3,0.0,NaN,NaN,NaN,NaN,NaN,None,None,None,0.000000
1,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,0.0,1.0,None,None,None,0.000000
2,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,112.0,2.0,None,None,None,0.000000
3,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None,0.000000
4,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.694267e+09,0.0,1.0,None,None,None,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111233,user_127750,male,11,578.0,1.692981e+09,1.692981e+09,1.692981e+09,0.0,1.0,None,None,None,1.358865
2111234,user_127750,male,11,371.0,1.691270e+09,1.691270e+09,1.691270e+09,32.0,4.0,[item_5409132],None,None,1.472542
2111235,user_127751,male,11,3.0,1.697627e+09,1.697627e+09,1.697627e+09,0.0,1.0,None,None,None,0.000000
2111236,user_127751,male,11,223.0,1.697627e+09,1.697627e+09,1.697627e+09,5.0,2.0,None,None,None,0.252460


In [39]:
df_visits_train['female_site_id'] = df_visits_train['site-id'].apply(lambda x: female_site_id_stat[x] if x in female_site_id_stat else 0.0)
df_visits_train

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands,male_site_id,female_site_id
0,user_1,female,3,0.0,NaN,NaN,NaN,NaN,NaN,None,None,None,0.000000,0.000000
1,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,0.0,1.0,None,None,None,0.000000,0.000000
2,user_2,female,11,3.0,1.696278e+09,1.696278e+09,1.696278e+09,112.0,2.0,None,None,None,0.000000,0.000000
3,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.693797e+09,0.0,1.0,None,None,None,0.000000,0.000000
4,user_5,female,35,3.0,1.693797e+09,1.698664e+09,1.694267e+09,0.0,1.0,None,None,None,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111233,user_127750,male,11,578.0,1.692981e+09,1.692981e+09,1.692981e+09,0.0,1.0,None,None,None,1.358865,0.855800
2111234,user_127750,male,11,371.0,1.691270e+09,1.691270e+09,1.691270e+09,32.0,4.0,[item_5409132],None,None,1.472542,1.741820
2111235,user_127751,male,11,3.0,1.697627e+09,1.697627e+09,1.697627e+09,0.0,1.0,None,None,None,0.000000,0.000000
2111236,user_127751,male,11,223.0,1.697627e+09,1.697627e+09,1.697627e+09,5.0,2.0,None,None,None,0.252460,0.141308


In [49]:
df_visits_val['male_site_id'] = df_visits_val['site-id'].apply(lambda x: male_site_id_stat[x] if x in male_site_id_stat else 0.0)
df_visits_val

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands,male_site_id
0,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,None,None,None,0.000000
1,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697133e+09,0.0,1.0,None,None,None,0.000000
2,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698855e+09,626.0,3.0,None,None,None,0.358051
3,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698931e+09,0.0,1.0,None,None,None,0.358051
4,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698947e+09,0.0,1.0,None,None,None,0.358051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,None,None,None,3.292980
561074,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,None,None,None,3.292980
561075,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,None,None,None,0.000000
561076,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,None,None,None,0.000000


In [50]:
df_visits_val['female_site_id'] = df_visits_val['site-id'].apply(lambda x: female_site_id_stat[x] if x in female_site_id_stat else 0.0)
df_visits_val

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands,male_site_id,female_site_id
0,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,None,None,None,0.000000,0.000000
1,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697133e+09,0.0,1.0,None,None,None,0.000000,0.000000
2,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698855e+09,626.0,3.0,None,None,None,0.358051,0.828761
3,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698931e+09,0.0,1.0,None,None,None,0.358051,0.828761
4,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698947e+09,0.0,1.0,None,None,None,0.358051,0.828761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,None,None,None,3.292980,3.638928
561074,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,None,None,None,3.292980,3.638928
561075,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,None,None,None,0.000000,0.000000
561076,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,None,None,None,0.000000,0.000000


In [44]:
# Функции для извлечения фичей из предобработанных файлов

def get_features_from_orders(x):
    features = {}
    features['length_mean'] = x['length'].mean()
    features['order_site-id_count'] = x['site-id'].count()
    features['order_site-id_unique'] = x['site-id'].nunique()
    #features['site-id_mode'] = x['site-id'].mode().tolist()#.to_string(index=False)
    features['id_count'] = x['id'].count()
    features['id_count_unique'] = x['id'].nunique()
    #features['id_mode'] = x['id'].mode().tolist()#.to_string(index=False)
    features['count_sum'] = x['count'].sum()
    #features['brand-id_mode'] = x['brand-id'].mode().tolist()#.to_string(index=False) 
    #features['brand-id_unique'] = x['brand-id'].nunique()
    return pd.Series(features, index=list(features.keys()))

def get_features_from_visits(x):
    features = {}
    features['session-duration_mean'] = x['session-duration'].mean()
    features['pages-count_mean'] = x['pages-count'].mean()
    features['visit_site-id_count'] = x['site-id'].count()
    features['visit_site-id_unique'] = x['site-id'].nunique()
    features['visit_site-id_mean_male'] = x['male_site_id'].mean()
    features['visit_site-id_mean_female'] = x['female_site_id'].mean()
    return pd.Series(features, index=list(features.keys()))

def extract_features(orders_df, visits_df):
    res_df = orders_df.groupby(['user_id', 'target']).apply(get_features_from_orders)
    res_df = res_df.reset_index()
    res_df = res_df.sort_values(by=['user_id'], key=lambda s: s.str[5:].astype(int))
    res_df = res_df.reset_index().drop(['index'], axis=1)
    res_df = res_df.replace('Series([], )', np.nan)
    
    visits_features = visits_df.groupby(['user_id']).apply(get_features_from_visits)
    visits_features = visits_features.reset_index()
    res_df = res_df.merge(visits_features, left_on='user_id', right_on='user_id')
    res_df = res_df.fillna(0)
    
    return res_df

In [45]:
train_df = extract_features(df_orders_train, df_visits_train)
train_df

,user_id,target,length_mean,order_site-id_count,order_site-id_unique,id_count,id_count_unique,count_sum,session-duration_mean,pages-count_mean,visit_site-id_count,visit_site-id_unique,visit_site-id_mean_male,visit_site-id_mean_female
0,user_1,female,3.0,3.0,2.0,0.0,0.0,0.0,0.000000,0.000000,1.0,1.0,0.000000,0.000000
1,user_2,female,11.0,0.0,0.0,0.0,0.0,0.0,56.000000,1.500000,2.0,1.0,0.000000,0.000000
2,user_3,male,16.0,2.0,2.0,1.0,1.0,1.0,195.928571,4.571429,14.0,3.0,5.239629,3.303693
3,user_4,male,27.0,9.0,4.0,4.0,4.0,4.0,267.764706,3.176471,17.0,6.0,3.041649,1.933425
4,user_5,female,35.0,6.0,3.0,5.0,5.0,5.0,279.391304,5.391304,23.0,5.0,6.098808,4.532126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119546,user_127751,male,11.0,5.0,2.0,3.0,3.0,3.0,2.500000,1.500000,2.0,2.0,0.126230,0.070654
119547,user_127752,male,13.0,3.0,2.0,1.0,1.0,1.0,0.000000,1.000000,1.0,1.0,0.000000,0.000000
119548,user_127753,female,24.0,17.0,1.0,17.0,12.0,20.0,106.600000,2.428571,35.0,8.0,3.345009,2.570047
119549,user_127754,female,8.0,1.0,1.0,1.0,1.0,0.0,101.000000,2.250000,4.0,2.0,0.268538,0.621570


In [46]:
train_df.to_parquet('data\\train_df.parquet')

In [51]:
val_df = extract_features(df_orders_val, df_visits_val)
val_df

,user_id,target,length_mean,order_site-id_count,order_site-id_unique,id_count,id_count_unique,count_sum,session-duration_mean,pages-count_mean,visit_site-id_count,visit_site-id_unique,visit_site-id_mean_male,visit_site-id_mean_female
0,user_127756,female,8.0,1.0,1.0,1.0,1.0,0.0,95.833333,1.583333,12.0,2.0,0.298376,0.690634
1,user_127757,male,16.0,16.0,4.0,14.0,14.0,16.0,621.000000,4.000000,3.0,3.0,2.078258,1.449996
2,user_127758,female,50.0,38.0,5.0,38.0,35.0,67.0,179.884615,4.365385,52.0,15.0,0.802134,0.860341
3,user_127759,female,34.0,52.0,3.0,52.0,46.0,52.0,105.314286,2.657143,35.0,9.0,0.368308,0.971747
4,user_127760,female,12.0,0.0,0.0,0.0,0.0,0.0,296.000000,18.333333,3.0,2.0,0.378047,0.902735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27442,user_155198,female,22.0,9.0,6.0,7.0,6.0,9.0,0.000000,1.000000,2.0,1.0,0.000000,0.000000
27443,user_155199,male,7.0,3.0,1.0,0.0,0.0,0.0,15.333333,1.333333,3.0,2.0,0.491161,0.373947
27444,user_155200,female,23.0,2.0,1.0,2.0,2.0,2.0,89.307692,1.846154,13.0,5.0,0.545640,0.870649
27445,user_155201,male,27.0,3.0,2.0,0.0,0.0,0.0,99.818182,2.272727,22.0,11.0,1.273613,1.450829


In [52]:
val_df.to_parquet('data\\val_df.parquet')

In [81]:
val_orders_grouped = val_orders_flat.groupby(['user_id', 'target']).apply(get_features_from_orders)
val_orders_grouped = val_orders_grouped.reset_index()
val_orders_grouped = val_orders_grouped.replace('Series([], )', np.nan)
val_orders_grouped

,user_id,target,site-id_count,site-id_unique,site-id_mode,id_count,id_count_unique,id_mode,count_sum,brand-id_mode,brand-id_unique
0,user_127756,female,1,1,[407.0],1,1,[item_676374],0.0,[],0
1,user_127757,male,16,4,[16.0],14,14,"[item_1399130, item_1627033, item_1898950, ite...",16.0,"[1055.0, 1237.0]",6
2,user_127758,female,38,5,[21.0],38,35,"[item_2072631, item_3362928, item_611359]",67.0,[15820.0],21
3,user_127759,female,52,3,[93.0],52,46,"[item_47281, item_47282]",52.0,[3192.0],3
4,user_127760,female,0,0,[],0,0,[],0.0,[],0
...,...,...,...,...,...,...,...,...,...,...,...
27442,user_155198,female,9,6,"[7.0, 8.0, 49.0]",7,6,[item_51490],9.0,[1889758.0],3
27443,user_155199,male,3,1,[21.0],0,0,[],0.0,[],0
27444,user_155200,female,2,1,[34.0],2,2,"[item_1648, item_6173373]",2.0,[708.0],1
27445,user_155201,male,3,2,[218.0],0,0,[],0.0,[],0
